Computo de alto desempeño
======================

Semana 2
Mayo 10, 2017

**Horacio Tapia** 
(htapia@lania.edu.mx)

LANIA-MCA, 2017

## Aproximando $\pi$ usando Monte Carlo

In [12]:
import numpy as np

In [14]:
np.pi

3.141592653589793

# Primeros pasos
Primero debemos importar el modulo y crear una instancia `Client`:

In [8]:
import ipyparallel as ipp
c = ipp.Client()

## Motores conectados al controlador 
Podemos ver la lista de ids de los motores disponibles. Esta debe coincidir con el numero especificado en la pestana. 

In [9]:
c.ids

[0, 1, 2, 3]

In [10]:
dview = c.direct_view() # es equivalente a dview = c[:]

### Funcion para aproximar $\pi$ (Monte Carlo)

In [16]:
def mc_pi(n):
    count = 0.
    for i in range(n):
        x = np.random.random()
        y = np.random.random()
        if (x**2+y**2)<=1:
            count += 1
    return count/n

In [17]:
%%timeit -n1
est_pi = mc_pi(10**6)
print 4*est_pi

3.13832
3.142812
3.141136
1 loop, best of 3: 744 ms per loop


Tratemos de paralelizarlo directamete usando `map_sync`

In [20]:
dview.map_sync(mc_pi,[250000]*4)

CompositeError: one or more exceptions from call to method: mc_pi
[Engine Exception]NameError: global name 'np' is not defined
[Engine Exception]NameError: global name 'np' is not defined
[Engine Exception]NameError: global name 'np' is not defined
[Engine Exception]NameError: global name 'np' is not defined

El error indica que `np` no esta definido. Sin embargo el modulo esta cargado en la sesion principal (el controlador):

In [15]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

?????

Resulta que cada motor pueden pensarse como sesiones independientes de Python. Aun cuando el modulo exita en el controlador, no existe en los motores. Tenemos que explicitamente importarlo:

In [24]:
with dview.sync_imports():
    import numpy

¿Como tendriamos que modificar nuestra funcion?

In [27]:
def mc_pi(n):
    count = 0.
    for i in range(n):
        x = np.random.random()
        y = np.random.random()
        if (x**2+y**2)<=1:
            count += 1
    return count/n

El error indica cual es el cambio que tenemos que realizar

In [28]:
%timeit -n1 dview.map_sync(mc_pi,[250000]*4)

1 loop, best of 3: 224 ms per loop


Tambien es posible importar el modulo dentro de la funcion:

In [212]:
def mc_pi_1(n):
    import numpy as np
    count = 0.
    for i in range(n):
        x = np.random.random()
        y = np.random.random()
        if (x**2+y**2)<=1:
            count += 1
    return count/n

In [213]:
dview.map_sync(mc_pi_1,[250000]*4)

[0.78554, 0.785084, 0.7858, 0.784172]

# Tarea

Considerar los siguientes codigos que aproximan el valor de $\pi$ usando el metodo de Monte Carlo:

In [75]:
def mc_pi_serial(n):
    count = 0.
    for i in range(n):
        x = np.random.random()
        y = np.random.random()
        if (x**2+y**2)<=1:
            count += 1
    return count/n

def mc_pi_serial_map(n):
    xy=np.random.random((n,2))
    dts=map(np.linalg.norm,xy)
    return np.sum(np.count_nonzero(np.less(dts,1.)))/np.float(n)

def mc_pi_numpy(n):
    xy=np.random.random((n,2))
    dts=np.linalg.norm(xy,axis=1)
    return np.sum(np.count_nonzero(np.less(dts,1.)))/np.float(n)

y sus respectivos tiempos de ejecucion

In [76]:
%%timeit -n1
est_pi = mc_pi_serial(10**6)
print 4*est_pi

3.141228
3.141288
3.140432
1 loop, best of 3: 467 ms per loop


In [77]:
%%timeit -n1
est_pi = mc_pi_serial_map(10**6)
print 4*est_pi

3.1395
3.142032
3.143472
1 loop, best of 3: 5.02 s per loop


In [78]:
%%timeit -n1
est_pi = mc_pi_numpy(10**6)
print 4*est_pi

3.141584
3.140684
3.1419
1 loop, best of 3: 40 ms per loop


1. En cada uno de ellos identificar la parte del codigo con el mayor costo computacional
2. Explicar porque el tiempo de ejecucion de cada uno es significativamente distinto
3. Paralelizar cada uno de ellos directamente usando `DirectView` y comparar los resultados entre ellos, cuantificando el resultado del tiempo de ejecucion como funcion del numero de motores usados